# Query by Donor Segment #

## Overview ##

This approach will create the following query contexts:

* `ctx_dseg`
* `ctx_dseg_memb`
* `ctx_donor`
* `ctx_indiv`
* `ctx_contrib`

## Notebook Setup ##

* Configure database connect information and options
* Clear potentially interfering context (PostgreSQL doesn't let you replace a view definition with conflicting column names)
* Set styling for notebook

In [1]:
sqlconnect = "postgresql+psycopg2://crash@localhost/fecdb"

%load_ext sql
%config SqlMagic.autopandas=True
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
%sql $sqlconnect

'Connected: crash@fecdb'

In [2]:
%sql drop view if exists ctx_contrib   cascade
%sql drop view if exists ctx_indiv     cascade
%sql drop view if exists ctx_donor     cascade
%sql drop view if exists ctx_iseg_memb cascade
%sql drop view if exists ctx_iseg      cascade
%sql drop view if exists ctx_dseg_memb cascade
%sql drop view if exists ctx_dseg      cascade

 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [3]:
%%html
<style>
  tr, th, td {
    text-align: left !important;
  }
</style>

## Create Donor Segments (Sample by Total Contribution Since 2000) ##

In [4]:
%sql delete from donor_seg
%sql drop table if exists seg_def
%sql drop materialized view if exists donor_sum
%sql drop materialized view if exists donor_sum_314
%sql drop materialized view if exists indiv_group

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [5]:
%%sql
create materialized view indiv_group as
select ip.part1                  as last_name,
       substr(ip.part2, 1, 3)    as first_name_pfx,
       substr(ip.zip_code, 1, 3) as zip_pfx,
       count(distinct ip.id)     as indivs,
       array_agg(distinct ip.id) as indiv_ids
  from indiv_parsed ip
 where ip.name ~ '^[A-Z][^,]'
   and ip.zip_code is not null
   and ip.num_parts > 1
   and ip.part1 !~ ' '
 group by 1, 2, 3

 * postgresql+psycopg2://crash@localhost/fecdb
6738578 rows affected.


""


Not sure whether it is better to re-aggregate the unnested ids (even though we are not able to omit the `distinct` qualifier), or select `ig.indiv_ids` and add to GROUP BY clause&mdash;voting for the former option right now...

In [6]:
%%sql
create materialized view donor_sum as
with indiv_group_memb as (
    select ig.last_name,
           ig.first_name_pfx,
           ig.zip_pfx,
           --ig.indiv_ids,
           unnest(ig.indiv_ids) as indiv_id
      from indiv_group ig
)
select igm.last_name,
       igm.first_name_pfx,
       igm.zip_pfx,
       array_agg(distinct igm.indiv_id)
                                 as indiv_ids,
       count(ic.transaction_amt) as contribs,
       sum(ic.transaction_amt)   as total_amt,
       round(sum(ic.transaction_amt) / count(ic.transaction_amt), 2)
                                 as avg_amt
  from indiv_group_memb igm
  join indiv_contrib ic on ic.indiv_id = igm.indiv_id
 group by 1, 2, 3

 * postgresql+psycopg2://crash@localhost/fecdb
6738578 rows affected.


""


In [7]:
%sql create index donor_sum_total_amt on donor_sum (total_amt)
%sql create index donor_sum_avg_amt   on donor_sum (avg_amt)

 * postgresql+psycopg2://crash@localhost/fecdb
Done.
 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [8]:
%%sql result <<
select count(*) as num_donors,
       sum(total_amt) as sum_total_amt
  from donor_sum

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.
Returning data to local variable result


In [9]:
(num_donors, sum_total_amt) = result.loc[0]
num_donors = int(num_donors)  # cast from numpy.int64

6738578

In [10]:
%%sql
select to_char(:num_donors,    '999,999,999')        as num_donors,
       to_char(:sum_total_amt, '999,999,999,999.99') as sum_total_amt

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.


,num_donors,sum_total_amt
0,"6,738,578","27,462,347,087.00"


In [11]:
%%sql
create temporary table seg_def as
select unnest(array[100000000,
                    50000000,
                    10000000,
                    5000000,
                    1000000,
                    500000,
                    100000,
                    50000,
                    10000,
                    5000,
                    2500,
                    1000,
                    500,
                    250,
                    0]) as seg_amt

 * postgresql+psycopg2://crash@localhost/fecdb
15 rows affected.


""


In [12]:
%%sql
select seg_def.seg_amt,
       seg_stat.donors,
       seg_stat.pct_donors,
       seg_stat.pct_total_amt
  from seg_def
  join lateral
       (select count(*)                                           as donors,
               round(count(*)::numeric / :num_donors * 100, 2)    as pct_donors,
               round(sum(ds.total_amt) / :sum_total_amt * 100, 1) as pct_total_amt
          from donor_sum ds
         where ds.total_amt > seg_def.seg_amt) as seg_stat on true

 * postgresql+psycopg2://crash@localhost/fecdb
15 rows affected.


,seg_amt,donors,pct_donors,pct_total_amt
0,100000000,4,0.00,2.7
1,50000000,10,0.00,4.2
2,10000000,63,0.00,7.9
3,5000000,152,0.00,10.2
4,1000000,1272,0.02,17.4
5,500000,3413,0.05,22.7
6,100000,27132,0.40,39.5
7,50000,61836,0.92,48.2
8,10000,364585,5.41,70.4
9,5000,713220,10.58,79.4


Utility function to help with naming of segments

In [13]:
%%sql
CREATE OR REPLACE FUNCTION human_readable(label TEXT) RETURNS TEXT AS $$
    SELECT regexp_replace(
               regexp_replace(
                   regexp_replace(
                       label,
                       '0{9}([^0-9]|$)', 'B\1'),
                   '0{6}([^0-9]|$)', 'M\1'),
               '0{3}([^0-9]|$)', 'K\1');
$$ LANGUAGE SQL

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


Now we create donor segments for sample records within each segment, and also connect the `indiv` records for each donor to a base indivual record

In [14]:
%%sql
CREATE OR REPLACE FUNCTION create_donor_seg_by_amt(seg_amt NUMERIC, seg_size INTEGER = 100)
RETURNS TABLE(seg_id BIGINT, seg_name TEXT) AS $$
DECLARE
sql      TEXT;
seg_name TEXT;
seg_desc TEXT = NULL;
BEGIN
    sql = format('select human_readable(concat(%L, $1, %L))', '$', '+ donors');
    EXECUTE sql INTO seg_name USING seg_amt;

    sql = 'with donor_set as (
               select row(ds.indiv_ids)::id_array as ids
                 from donor_sum ds
                where ds.total_amt > $2
                order by ds.total_amt asc
                limit ($3)
           )
           select create_donor_seg(array_agg(ids), $1),
                  $1
             from donor_set
            group by 2';
    RETURN QUERY EXECUTE sql USING seg_name, seg_amt, seg_size;
END;
$$ LANGUAGE plpgsql

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [15]:
%%sql
select seg_def.seg_amt, (create_donor_seg_by_amt(seg_def.seg_amt)).*
  from seg_def

 * postgresql+psycopg2://crash@localhost/fecdb
15 rows affected.


,seg_amt,seg_id,seg_name
0,100000000,49,$100M+ donors
1,50000000,50,$50M+ donors
2,10000000,51,$10M+ donors
3,5000000,52,$5M+ donors
4,1000000,53,$1M+ donors
5,500000,54,$500K+ donors
6,100000,55,$100K+ donors
7,50000,56,$50K+ donors
8,10000,57,$10K+ donors
9,5000,58,$5K+ donors


In [16]:
%%sql
select ds.id    as seg_id,
       ds.name  as seg_name,
       count(*) as seg_members
  from donor_seg ds
  join donor_seg_memb dsm on dsm.donor_seg_id = ds.id
 group by 1, 2
 order by 1

 * postgresql+psycopg2://crash@localhost/fecdb
15 rows affected.


,seg_id,seg_name,seg_members
0,49,$100M+ donors,4
1,50,$50M+ donors,10
2,51,$10M+ donors,63
3,52,$5M+ donors,100
4,53,$1M+ donors,100
5,54,$500K+ donors,100
6,55,$100K+ donors,100
7,56,$50K+ donors,100
8,57,$10K+ donors,100
9,58,$5K+ donors,100


## Create Context Views ##

### Create `ctx_dseg` ###

In [17]:
%%sql
create or replace view ctx_dseg as
select id,
       name,
       description
  from donor_seg ds
 where ds.name = '$50M+ donors'

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [18]:
%%sql
select *
  from ctx_dseg

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.


,id,name,description
0,50,$50M+ donors,None


In [19]:
%%sql
with seg_donors as (
    select dsm.donor_indiv_id
      from ctx_dseg dsx
      join donor_seg_memb dsm on dsm.donor_seg_id = dsx.id
)
select i.id,
       i.name,
       i.city,
       i.state,
       i.zip_code
  from seg_donors sd
  join indiv i on i.id = sd.donor_indiv_id
 order by i.name, i.zip_code

 * postgresql+psycopg2://crash@localhost/fecdb
10 rows affected.


,id,name,city,state,zip_code
0,70692,"ADELSON, MIRIAM",LAS VEAGS,NV,89109
1,70790,"ADELSON, SHELDON",LAS VEGAS,NV,89109
2,1076630,"BLOOMBERG, MICHAEL",NEW YORK,NY,10021
3,3535137,"EYCHANER, FRED",CHICAGO,IL,60614
4,7961557,"MCMAHON, LINDA",STAMFORD,CT,06901
5,10804004,"SCOTT, RICK",TALLAHASSEE,FL,32315
6,11121369,"SIMONS, JAMES",NEW YORK,NY,10010
7,11141923,"SINGER, PAUL",NEW YOR,NY,10024
8,11612429,"STEYER, THOMAS",SAN FRANCISCO,CA,94104
9,12286548,"UIHLEIN, RICHARD",LAKE BLUFF,IL,60044


### Create `ctx_dseg_memb` ###

In [20]:
%%sql
create or replace view ctx_dseg_memb as
select dsm.*
  from ctx_dseg dsx
  join donor_seg_memb dsm on dsm.donor_seg_id = dsx.id

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [21]:
%%sql
select ds.name as dseg_name,
       d.name  as indiv_name,
       d.city,
       d.state,
       d.zip_code,
       d.elect_cycles
  from ctx_dseg_memb dsmx
  join donor_seg ds on ds.id = dsmx.donor_seg_id
  join donor_indiv d on d.id = dsmx.donor_indiv_id

 * postgresql+psycopg2://crash@localhost/fecdb
10 rows affected.


,dseg_name,indiv_name,city,state,zip_code,elect_cycles
0,$50M+ donors,"SINGER, PAUL",NEW YOR,NY,10024,"[2016, 2018]"
1,$50M+ donors,"SIMONS, JAMES",NEW YORK,NY,10010,"[2010, 2012, 2014, 2016, 2018]"
2,$50M+ donors,"SCOTT, RICK",TALLAHASSEE,FL,32315,[2018]
3,$50M+ donors,"UIHLEIN, RICHARD",LAKE BLUFF,IL,60044,"[2000, 2002]"
4,$50M+ donors,"MCMAHON, LINDA",STAMFORD,CT,06901,"[2010, 2012, 2014]"
5,$50M+ donors,"EYCHANER, FRED",CHICAGO,IL,60614,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
6,$50M+ donors,"ADELSON, MIRIAM",LAS VEAGS,NV,89109,[2014]
7,$50M+ donors,"BLOOMBERG, MICHAEL",NEW YORK,NY,10021,"[2000, 2004, 2006, 2010]"
8,$50M+ donors,"ADELSON, SHELDON",LAS VEGAS,NV,89109,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
9,$50M+ donors,"STEYER, THOMAS",SAN FRANCISCO,CA,94104,"[2014, 2016, 2018]"


### Create `ctx_donor` ###

In [22]:
%%sql
create or replace view ctx_donor as
select d.*
  from ctx_dseg_memb dsmx
  join donor_indiv d on d.id = dsmx.donor_indiv_id

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [23]:
%%sql
select id,
       name,
       city,
       state,
       zip_code,
       elect_cycles
  from ctx_donor

 * postgresql+psycopg2://crash@localhost/fecdb
10 rows affected.


,id,name,city,state,zip_code,elect_cycles
0,11141923,"SINGER, PAUL",NEW YOR,NY,10024,"[2016, 2018]"
1,11121369,"SIMONS, JAMES",NEW YORK,NY,10010,"[2010, 2012, 2014, 2016, 2018]"
2,10804004,"SCOTT, RICK",TALLAHASSEE,FL,32315,[2018]
3,12286548,"UIHLEIN, RICHARD",LAKE BLUFF,IL,60044,"[2000, 2002]"
4,7961557,"MCMAHON, LINDA",STAMFORD,CT,06901,"[2010, 2012, 2014]"
5,3535137,"EYCHANER, FRED",CHICAGO,IL,60614,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
6,70692,"ADELSON, MIRIAM",LAS VEAGS,NV,89109,[2014]
7,1076630,"BLOOMBERG, MICHAEL",NEW YORK,NY,10021,"[2000, 2004, 2006, 2010]"
8,70790,"ADELSON, SHELDON",LAS VEGAS,NV,89109,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."
9,11612429,"STEYER, THOMAS",SAN FRANCISCO,CA,94104,"[2014, 2016, 2018]"


### Create `ctx_indiv` ###

In [24]:
%%sql
create or replace view ctx_indiv as
select i.*
  from ctx_donor dx
  join indiv i on i.donor_indiv_id = dx.id

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [25]:
%%sql
select id,
       name,
       city,
       state,
       zip_code,
       elect_cycles,
       donor_indiv_id
  from ctx_indiv
 order by donor_indiv_id, name

 * postgresql+psycopg2://crash@localhost/fecdb
295 rows affected.


,id,name,city,state,zip_code,elect_cycles,donor_indiv_id
0,70693,"ADELSON, MIRIAM",LAS VEGAS,NV,891016200,[2018],70692
1,70699,"ADELSON, MIRIAM",LAS VEGAS,NV,89169,[2012],70692
2,70698,"ADELSON, MIRIAM",LAS VEGAS,NV,891455749,"[2016, 2018, 2020]",70692
3,70700,"ADELSON, MIRIAM",LAS VEGAS,NY,89145,"[2014, 2016]",70692
4,70692,"ADELSON, MIRIAM",LAS VEAGS,NV,89109,[2014],70692
...,...,...,...,...,...,...,...
290,12286604,"UIHLEIN, RICHARD MR.",LAKE FOREST,IL,600451147,"[2014, 2016, 2018, 2020]",12286548
291,12286602,"UIHLEIN, RICHARD MR.",LAKE BLUFF,IL,60044,[2002],12286548
292,12286605,"UIHLEIN, RICHARD MR.",LAKE FORST,IL,60045,[2018],12286548
293,12286603,"UIHLEIN, RICHARD MR.",LAKE FOREST,IL,60045,"[2000, 2002, 2008, 2010, 2012, 2016, 2018, 2020]",12286548


### Create `ctx_contrib` ###

In [26]:
%%sql
create or replace view ctx_contrib as
select ic.*,
       ix.donor_indiv_id
  from ctx_indiv ix
  join indiv_contrib ic on ic.indiv_id = ix.id

 * postgresql+psycopg2://crash@localhost/fecdb
Done.


""


In [27]:
%%sql
select count(*)             as contribs,
       sum(transaction_amt) as total_amt,
       array_agg(distinct elect_cycle) as elect_cycles
  from ctx_contrib

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.


,contribs,total_amt,elect_cycles
0,4932,1140215466.00,"[2000, 2002, 2004, 2006, 2008, 2010, 2012, 201..."


## Query Based on Context ##

### Query using `ctx_dseg` ###

In [28]:
%%sql
select ic.elect_cycle,
       count(*) cycle_contribs,
       sum(ic.transaction_amt) cycle_amount,
       round(avg(ic.transaction_amt), 2) avg_amount,
       min(ic.transaction_amt) min_amount,
       max(ic.transaction_amt) max_amount
  from ctx_dseg dsx
  join donor_seg_memb dsm on dsm.donor_seg_id = dsx.id
  join indiv i on i.donor_indiv_id = dsm.donor_indiv_id
  join indiv_contrib ic on ic.indiv_id = i.id
 group by 1
 order by 1

 * postgresql+psycopg2://crash@localhost/fecdb
11 rows affected.


,elect_cycle,cycle_contribs,cycle_amount,avg_amount,min_amount,max_amount
0,2000,181,1474250.00,8145.03,250.00,150000.00
1,2002,120,6938713.00,57822.61,-1000.00,3000000.00
2,2004,167,3118846.00,18675.72,-2000.00,500000.00
3,2006,203,758300.00,3735.47,-500.00,26700.00
4,2008,201,1041995.00,5184.05,-2300.00,107000.00
5,2010,397,54825025.00,138098.30,-2400.00,6000000.00
6,2012,414,150537145.00,363616.29,-500000.00,5000000.00
7,2014,614,144052734.00,234613.57,-10200.00,16000000.00
8,2016,1070,312333725.00,291900.68,-226800.00,11000000.00
9,2018,1295,437046590.00,337487.71,-10000.00,20000000.00


### Query using `ctx_dseg_memb` ###

In [29]:
%%sql
select ic.elect_cycle,
       count(*) cycle_contribs,
       sum(ic.transaction_amt) cycle_amount,
       round(avg(ic.transaction_amt), 2) avg_amount,
       min(ic.transaction_amt) min_amount,
       max(ic.transaction_amt) max_amount
  from ctx_dseg_memb dsmx
  join indiv i on i.donor_indiv_id = dsmx.donor_indiv_id
  join indiv_contrib ic on ic.indiv_id = i.id
 group by 1
 order by 1

 * postgresql+psycopg2://crash@localhost/fecdb
11 rows affected.


,elect_cycle,cycle_contribs,cycle_amount,avg_amount,min_amount,max_amount
0,2000,181,1474250.00,8145.03,250.00,150000.00
1,2002,120,6938713.00,57822.61,-1000.00,3000000.00
2,2004,167,3118846.00,18675.72,-2000.00,500000.00
3,2006,203,758300.00,3735.47,-500.00,26700.00
4,2008,201,1041995.00,5184.05,-2300.00,107000.00
5,2010,397,54825025.00,138098.30,-2400.00,6000000.00
6,2012,414,150537145.00,363616.29,-500000.00,5000000.00
7,2014,614,144052734.00,234613.57,-10200.00,16000000.00
8,2016,1070,312333725.00,291900.68,-226800.00,11000000.00
9,2018,1295,437046590.00,337487.71,-10000.00,20000000.00


### Query using `ctx_indiv` ###

In [30]:
%%sql
select ic.elect_cycle,
       count(*) cycle_contribs,
       sum(ic.transaction_amt) cycle_amount,
       round(avg(ic.transaction_amt), 2) avg_amount,
       min(ic.transaction_amt) min_amount,
       max(ic.transaction_amt) max_amount
  from ctx_indiv ix
  join indiv_contrib ic on ic.indiv_id = ix.id
 group by 1
 order by 1

 * postgresql+psycopg2://crash@localhost/fecdb
11 rows affected.


,elect_cycle,cycle_contribs,cycle_amount,avg_amount,min_amount,max_amount
0,2000,181,1474250.00,8145.03,250.00,150000.00
1,2002,120,6938713.00,57822.61,-1000.00,3000000.00
2,2004,167,3118846.00,18675.72,-2000.00,500000.00
3,2006,203,758300.00,3735.47,-500.00,26700.00
4,2008,201,1041995.00,5184.05,-2300.00,107000.00
5,2010,397,54825025.00,138098.30,-2400.00,6000000.00
6,2012,414,150537145.00,363616.29,-500000.00,5000000.00
7,2014,614,144052734.00,234613.57,-10200.00,16000000.00
8,2016,1070,312333725.00,291900.68,-226800.00,11000000.00
9,2018,1295,437046590.00,337487.71,-10000.00,20000000.00


### Query using `ctx_contrib` ###

In [31]:
%%sql
select cx.elect_cycle,
       count(*) cycle_contribs,
       sum(cx.transaction_amt) cycle_amount,
       round(avg(cx.transaction_amt), 2) avg_amount,
       min(cx.transaction_amt) min_amount,
       max(cx.transaction_amt) max_amount
  from ctx_contrib cx
 group by 1
 order by 1

 * postgresql+psycopg2://crash@localhost/fecdb
11 rows affected.


,elect_cycle,cycle_contribs,cycle_amount,avg_amount,min_amount,max_amount
0,2000,181,1474250.00,8145.03,250.00,150000.00
1,2002,120,6938713.00,57822.61,-1000.00,3000000.00
2,2004,167,3118846.00,18675.72,-2000.00,500000.00
3,2006,203,758300.00,3735.47,-500.00,26700.00
4,2008,201,1041995.00,5184.05,-2300.00,107000.00
5,2010,397,54825025.00,138098.30,-2400.00,6000000.00
6,2012,414,150537145.00,363616.29,-500000.00,5000000.00
7,2014,614,144052734.00,234613.57,-10200.00,16000000.00
8,2016,1070,312333725.00,291900.68,-226800.00,11000000.00
9,2018,1295,437046590.00,337487.71,-10000.00,20000000.00


In [32]:
%%sql
select d.id as donor_id,
       d.name as donor_name,
       cx.elect_cycle,
       count(*) cycle_contribs,
       sum(cx.transaction_amt) cycle_amount,
       round(avg(cx.transaction_amt), 2) avg_amount,
       min(cx.transaction_amt) min_amount,
       max(cx.transaction_amt) max_amount
  from ctx_contrib cx
  join donor_indiv d on d.id = cx.donor_indiv_id
 group by 1, 2, 3
 order by 5 desc
 limit 50

 * postgresql+psycopg2://crash@localhost/fecdb
50 rows affected.


,donor_id,donor_name,elect_cycle,cycle_contribs,cycle_amount,avg_amount,min_amount,max_amount
0,1076630,"BLOOMBERG, MICHAEL",2018,56,95110857.00,1698408.16,-2700.00,20000000.00
1,11612429,"STEYER, THOMAS",2016,69,89988044.00,1304174.55,-2300.00,11000000.00
2,11612429,"STEYER, THOMAS",2014,27,78845867.00,2920217.30,867.00,16000000.00
3,11612429,"STEYER, THOMAS",2018,59,73089512.00,1238805.29,1000.00,13504144.00
4,10804004,"SCOTT, RICK",2018,169,69942510.00,413861.01,-100.00,7450000.00
5,70790,"ADELSON, SHELDON",2018,117,61971800.00,529673.50,-2700.00,15000000.00
6,70692,"ADELSON, MIRIAM",2018,117,61757400.00,527841.03,-2700.00,15000000.00
7,7961557,"MCMAHON, LINDA",2010,144,53107350.00,368801.04,21.00,6000000.00
8,70790,"ADELSON, SHELDON",2012,61,51023525.00,836451.23,-2500.00,5000000.00
9,70692,"ADELSON, MIRIAM",2012,50,46626900.00,932538.00,-500000.00,5000000.00
